<a href="https://colab.research.google.com/github/tronghieu2810/COMPUTER_VISION/blob/main/UDM_Lazy_Programmer_Inc_De_Le_Ad_Co_Vi/SSD_Tensorflow_2_Object_Detection_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Beginner Warning

This is dependent on a separate library which is not part of Tensorflow (but still made by the Tensorflow team).

Some beginners got "stuck" in the past when Tensorflow 2 came out, but this library was not compatible with Tensorflow 2.

Beginners often feel uncomfortable using older versions of libraries. This is *not* how things work in the real world.

See "Why bad programmers always need the latest version": https://youtu.be/BIXH_m6CT2I

## Clone Tensorflow Models Repository

In [1]:
!git clone https://github.com/tensorflow/models.git

Cloning into 'models'...
remote: Enumerating objects: 70382, done.
remote: Total 70382 (delta 0), reused 0 (delta 0), pack-reused 70382
Receiving objects: 100% (70382/70382), 578.36 MiB | 21.16 MiB/s, done.
Resolving deltas: 100% (49650/49650), done.


## Install Protobuf

No need to do this if you are on Colab.

If you're doing this locally, download the latest Protobuf library for your OS from https://github.com/google/protobuf/releases

The filename should look like "protoc-\*-\*.zip".

Example: protoc-3.17.3-win64.zip if you are using 64-bit Windows.

Assuming you've unzipped this zip file to \<path\>, the next step is to add \<path\>/bin to your PATH environment variable (on Linux or Mac).

Once Protobuf has been successfully installed, you can run the following command (note: must be done from the models/research folder).

In [2]:
!cd models/research/ && protoc object_detection/protos/*.proto --python_out=.

## Install the object detection API

In [3]:
!cd models/research && \
    cp object_detection/packages/tf2/setup.py . && \
    python -m pip install .

Processing /content/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
     |████████████████████████████████| 10.1 MB 15.8 MB/s 
     |████████████████████████████████| 352 kB 46.2 MB/s 
     |████████████████████████████████| 2.2 MB 34.5 MB/s 
     |████████████████████████████████| 23.4 MB 50.1 MB/s 
     |████████████████████████████████| 99 kB 7.3 MB/s 
     |████████████████████████████████| 90 kB 8.3 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 1.2 MB 35.3 MB/s 
     |████████████████████████████████| 636 kB 40.6 MB/s 
     |████████████████████████████████| 47.8 MB 48

## Test Installation (Optional)

In [ ]:
# !cd models/research && python object_detection/builders/model_builder_tf2_test.py

## Import Libraries

In [5]:
import os
import imageio
import pathlib
import tensorflow as tf
import time
from   object_detection.utils import label_map_util
from   object_detection.utils import visualization_utils as viz_utils
import numpy as np
from   PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

## Download Images

In [8]:
!wget -nc https://lazyprogrammer.me/cnn_class2_videos.zip
!unzip cnn_class2_videos.zip
!ls

--2022-03-16 23:23:39--  https://lazyprogrammer.me/cnn_class2_videos.zip
Resolving lazyprogrammer.me (lazyprogrammer.me)... 104.21.23.210, 172.67.213.166, 2606:4700:3030::ac43:d5a6, ...
Connecting to lazyprogrammer.me (lazyprogrammer.me)|104.21.23.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2073140 (2.0M) [application/zip]
Saving to: ‘cnn_class2_videos.zip’

cnn_class2_videos.z 100%[===================>]   1.98M  3.70MB/s    in 0.5s    

2022-03-16 23:23:40 (3.70 MB/s) - ‘cnn_class2_videos.zip’ saved [2073140/2073140]

Archive:  cnn_class2_videos.zip
  inflating: catdog.mp4              
  inflating: safari.mp4              
  inflating: traffic.mp4             
catdog.mp4  cnn_class2_videos.zip  models  safari.mp4  sample_data  traffic.mp4


In [9]:
INPUT_VIDEOS = ['catdog', 'safari', 'traffic']

## Download and extract model files

Get URLs from the "Object Detection Zoo": https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md

In [10]:
url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8.tar.gz'

# Download the model file
PATH_TO_MODEL_DIR = tf.keras.utils.get_file(fname='ssd_resnet101_v1_fpn_640x640_coco17_tpu-8', origin=url, untar=True)
print('')
PATH_TO_MODEL_DIR

386539520/386527459 [==============================] - 3s 0us/step



'/root/.keras/datasets/ssd_resnet101_v1_fpn_640x640_coco17_tpu-8'

## Download Labels File

Label files can be found here: https://github.com/tensorflow/models/tree/master/research/object_detection/data

You probably won't need these since Object Detection Zoo contains only models trained on COCO.

In [13]:
url = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/data/mscoco_label_map.pbtxt'

PATH_TO_LABELS = tf.keras.utils.get_file(fname='mscoco_label_map.pbtxt', origin=url, untar=False)
print('')
print(PATH_TO_LABELS)
print('')
!head {PATH_TO_LABELS}


/root/.keras/datasets/mscoco_label_map.pbtxt

item {
  name: "/m/01g317"
  id: 1
  display_name: "person"
}
item {
  name: "/m/0199g"
  id: 2
  display_name: "bicycle"
}


## Load in the model

In [15]:
PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + '/saved_model'

print('Loading model --- ', end='')
start_time = time.time()

# Load saved model & build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)
end_time = time.time()

elapsed_time = end_time - start_time
print(f'{elapsed_time} seconds.')

Loading model --- 39.59739828109741 seconds.


## Load in the labels

In [14]:
category_index = label_map_util.create_category_index_from_labelmap(label_map_path=PATH_TO_LABELS, use_display_name=True)
category_index

{1: {'id': 1, 'name': 'person'},
 2: {'id': 2, 'name': 'bicycle'},
 3: {'id': 3, 'name': 'car'},
 4: {'id': 4, 'name': 'motorcycle'},
 5: {'id': 5, 'name': 'airplane'},
 6: {'id': 6, 'name': 'bus'},
 7: {'id': 7, 'name': 'train'},
 8: {'id': 8, 'name': 'truck'},
 9: {'id': 9, 'name': 'boat'},
 10: {'id': 10, 'name': 'traffic light'},
 11: {'id': 11, 'name': 'fire hydrant'},
 13: {'id': 13, 'name': 'stop sign'},
 14: {'id': 14, 'name': 'parking meter'},
 15: {'id': 15, 'name': 'bench'},
 16: {'id': 16, 'name': 'bird'},
 17: {'id': 17, 'name': 'cat'},
 18: {'id': 18, 'name': 'dog'},
 19: {'id': 19, 'name': 'horse'},
 20: {'id': 20, 'name': 'sheep'},
 21: {'id': 21, 'name': 'cow'},
 22: {'id': 22, 'name': 'elephant'},
 23: {'id': 23, 'name': 'bear'},
 24: {'id': 24, 'name': 'zebra'},
 25: {'id': 25, 'name': 'giraffe'},
 27: {'id': 27, 'name': 'backpack'},
 28: {'id': 28, 'name': 'umbrella'},
 31: {'id': 31, 'name': 'handbag'},
 32: {'id': 32, 'name': 'tie'},
 33: {'id': 33, 'name': 'suitc

## Helper function

In [ ]:
## Helper function

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))

## Do some object detection

In [20]:
def detect_objects_in_image(image_np):
    # The input needs to be a Tensor
    input_tensor = tf.convert_to_tensor(image_np)

    # The model expects a batch of images (N * Height * Width * 3)
    input_tensor = input_tensor[tf.newaxis, ...]

    # Do the detection
    detections = detect_fn(input_tensor)

    # All outputs are batches tensors
    # Convert to Numpy arrays, take index [0] to remove the batch dimension
    # We're only interested in the first dimension
    num_detections = int(detections.pop('num_detections'))
    detections     = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # Detection_classes should be ints
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    # Show classes
    # unique_classes = set(detections['detection_classes']) # Cast a class array to a set
    # print('Classes found:')
    # for c in unique_classes:
    #     print(f"- {category_index[c]['name']}")
    
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
        image=image_np_with_detections,
        boxes=detections['detection_boxes'],
        classes=detections['detection_classes'],
        scores=detections['detection_scores'],
        category_index=category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=200,
        min_score_thresh=0.30,
        agnostic_mode=False
    )

    return image_np_with_detections

In [22]:
def detect_objects_in_video(input_video):
    print(f"Running inference for '{input_video}.mp4'")

    video_reader = imageio.get_reader(f'{input_video}.mp4')
    video_writer = imageio.get_writer(f'{input_video}_annotated.mp4', fps=10)

    # Loop through and process each frame
    t0       = time.time()
    n_frames = 0

    for frame in video_reader:
        n_frames += 1
        new_frame = detect_objects_in_image(image_np=frame)

        # Instead of plotting image, we write the frame to video
        video_writer.append_data(new_frame)
    
    fps = n_frames / (time.time() - t0)
    print(f'Frames processed: {n_frames}, Speed: {round(fps, 2)}')

    # Clean up
    video_writer.close()

In [23]:
detect_objects_in_video(INPUT_VIDEOS[0])

Running inference for 'catdog.mp4'
Frames processed: 50, Speed: 2.13
